In [1]:
#This program loads the transaction data to the block table
# Libraries pandas, json, psycopg2, logging
# This program contains three steps:
#1. Connect to database, and verify connection
#2. Fetch file based on block number, and verify existence
#3. Pull desired data from data paths and push it to block table
import pandas as pd
import json
import psycopg2
import logging

# 1. Connect to database
DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'

# Verify connection through try and except block
try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file
# If db does not exist produce error message
except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')
        

Connection established.


In [2]:
import os
import logging
# 2. Fetch block number from blocks directory
block = '1199803' # Input desired block number

  # Change this number if you want to load a different file
file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'
# Verify existence with try and except block
try:
    if os.path.exists(file_path):
        print(f"File '{file_path}' exists. Proceeding with further actions.")
        # Add your additional code here

        # Configure logging to write to both log.txt and error.txt
        logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info(f"File '{file_path}' exists. Proceeding with further actions.")

    else:
        print(f"File '{file_path}' does not exist.")
        # Log the error message to error.txt
        with open('error.txt', 'a') as error_file:
            error_file.write(f"File '{file_path}' does not exist.\n")

except Exception as e:
    print(f"An error occurred: {e}")
    # Log the error message to error.txt
    with open('error.txt', 'a') as error_file:
        error_file.write(f"An error occurred: {e}\n")
        
        

File 'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/1199803.json' exists. Proceeding with further actions.


In [3]:
# 3. Pull data from data paths and push to block table
# Identify data paths 
try:
    with open(file_path, 'r') as json_file:
        json_data = json.load(json_file)

        type = json_data["tx"]["body"]["messages"][0]["@type"]
        hash = json_data["hashed_result"]
        table_name = 'block'
# Insert data to db
        # Construct the combined insert query
        tx_header_insert_query = f"""
        INSERT INTO block (type, block, hash)
        VALUES ('{type}', {block}, '{hash}');
    """

        # Replace with your actual database connection parameters
        conn = psycopg2.connect(
            dbname='blockchain',
            user='postgres',
            password='auth2020',
            host='localhost',
            port='5432'
        )

        with conn:
            with conn.cursor() as cur:
                cur.execute(tx_header_insert_query)

        print("Data inserted successfully.")

except (psycopg2.DatabaseError, Exception) as error:
    print("Error inserting data into the table.")
    print(error)

finally:
    # Close the cursor and database connection
    cur.close()
    conn.close()


Data inserted successfully.
